In [38]:
import pandas as pd
from datetime import datetime as dt
import pickle
import networkx as nx

from utils.build_functions import get_clean_wikipedia_hyperlinks
from utils.graph_functions import node_sizes_scaling,hv_plot_graph
from networkx.drawing.nx_agraph import graphviz_layout

In [2]:
allPresidentsPost1940=["Herbert Hoover","Franklin D. Roosevelt","Harry S. Truman","Dwight D. Eisenhower","John F. Kennedy",
            "Lyndon B. Johnson","Richard Nixon","Gerald Ford","Jimmy Carter","Ronald Reagan","George H. W. Bush",
            "Bill Clinton","George W. Bush","Barack Obama","Donald Trump","Joe Biden"]
Presidents=["Jimmy Carter","Ronald Reagan","George H. W. Bush",
            "Bill Clinton","George W. Bush","Barack Obama","Donald Trump","Joe Biden"]

date = dt.now().strftime('%d%b%Y')

# Senate

In [22]:
dfs = pd.read_pickle("pickled/US_Senate_2024_df_updated_18Jun2024.pickle")
print(len(dfs))
dfs

500


,candidate,incumbency,gender,party,office,status,State,Wiki page,Wiki page (validated)
0,Kari Lake,Challenger,,Republican,U.S. Senate Arizona,On the Ballot Primary,AZ,Kari Lake,Kari Lake
1,Mark Lamb,Challenger,,Republican,U.S. Senate Arizona,On the Ballot Primary,AZ,Mark Lamb,Mark Lamb (sheriff)
2,Elizabeth Reye,Challenger,,Republican,U.S. Senate Arizona,On the Ballot Primary,AZ,None,None
3,Dustin Williams,Challenger,,Republican,U.S. Senate Arizona,On the Ballot (Write-in) Primary,AZ,None,None
4,Ruben Gallego,Challenger,,Democratic,U.S. Senate Arizona,On the Ballot Primary,AZ,Ruben Gallego,Ruben Gallego
...,...,...,...,...,...,...,...,...,...
495,Joshua Harrington,Challenger,,No Party Affiliation,U.S. Senate Wisconsin,Withdrew General,WI,None,None
496,John Holtz,Challenger,,Republican,U.S. Senate Wyoming,On the Ballot Primary,WY,None,None
497,Scott Morrow,Challenger,,Democratic,U.S. Senate Wyoming,On the Ballot Primary,WY,Scott Morrow,None
498,John Barrasso,Incumbent,,Republican,U.S. Senate Wyoming,Candidacy Declared Primary,WY,John Barrasso,John Barrasso


### Scrape hyperlinks from validated candidate Wikipedia pages

In [25]:
p = dfs.loc[dfs['Wiki page (validated)'].notnull(), 'Wiki page (validated)'].tolist() + Presidents
p=sorted(set(p))

sp_hyperlinks_d={} # all hyperlinks in a candidate's wiki page
sp_hyperlinks_inside_d={} # hyperlinks to other candidates
for pp in p:
    try:
        ehp=get_clean_wikipedia_hyperlinks(pp) 
        ppp=sorted([w for w in ehp]) 
    except Error as e:
        pass
    ppp=sorted([x for x in ppp if x!=pp])
    sp_hyperlinks_d[pp]=ppp
    ppp_inside=sorted(set(ppp).intersection(set(p)))
    sp_hyperlinks_inside_d[pp]=ppp_inside

# print(len(sp_hyperlinks_d))
with open(f'pickled/sp_hyperlinks_d_{date}', 'wb') as f:
    pickle.dump(sp_hyperlinks_d, f)
# print(len(sp_hyperlinks_inside_d))
with open(f'pickled/sp_hyperlinks_inside_d_{date}', 'wb') as f:
    pickle.dump(sp_hyperlinks_inside_d, f)

84
84
84


### Build the hyperlink graph of US Senate candidates' wiki pages

In [29]:
edges = []
for k,v in sp_hyperlinks_inside_d.items():
    if len(v)>0:
        for w in v:
            edges.append((k,w))
print(len(edges),len(set(edges)))

Gsp = nx.DiGraph()
Gsp.add_edges_from(edges)

wp_d = dfs.set_index('Wiki page (validated)')['candidate'].to_dict()
Gsp = nx.relabel_nodes(Gsp, wp_d)

283 283


In [30]:
senate_candidate_incumbency_d = dict(zip(dfs['candidate'], dfs['incumbency']))
senate_candidate_party_d = dict(zip(dfs['candidate'], dfs['party']))
senate_candidate_state_d = dict(zip(dfs['candidate'], dfs['State']))
senate_candidate_status_d = dict(zip(dfs['candidate'], dfs['status']))
        
# Set attributes
s_attr_dicts = [senate_candidate_incumbency_d, senate_candidate_party_d, senate_candidate_state_d, senate_candidate_status_d]
attr_names = ["incumbency", "party", "state", "status"]
for attr_dict, attr_name in zip(s_attr_dicts, attr_names):
    nx.set_node_attributes(Gsp, attr_dict, attr_name)

# Pickle graph
with open(f'pickled/US_Senate_2024_graph_updated_{date}.pickle', 'wb') as f:
    pickle.dump(Gsp, f)